In [ ]:
!pip install -q aiohttp aiodns async-timeout requests beautifulsoup4 pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 641.1/641.1 kB 10.0 MB/s eta 0:00:00


In [ ]:
import asyncio
import aiohttp
from aiohttp import ClientSession
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

BASE = "https://books.toscrape.com/"

In [ ]:
import requests
base_url = "https://books.toscrape.com/"
resp = requests.get(base_url)
resp.raise_for_status()
html_text = resp.text

print("=== HTML Source of homepage (first 2000 chars) ===\n")
print(html_text[:2000])
print("\n... (HTML truncated for display) ...\n")

=== HTML Source of homepage (first 2000 chars) ===

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    All products | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
        <meta name="created" content="24th Jun 2016 09:29" />
        <meta name="description" content="" />
        <meta name="viewport" content="width=device-width" />
        <meta name="robots" content="NOARCHIVE,NOCACHE" />

        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
        <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->

        
            <

In [ ]:
book_links = []
page_url = urljoin(BASE, "catalogue/page-1.html")
import requests
while page_url:
    r = requests.get(page_url, timeout=15)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")
    for art in soup.select("article.product_pod"):
        a = art.select_one("h3 a")
        title = a["title"].strip() if a and a.has_attr("title") else a.get_text(strip=True)
        rel_link = a["href"]
        product_link = urljoin(page_url, rel_link)
        price = art.select_one("p.price_color").get_text(strip=True) if art.select_one("p.price_color") else ""
        availability = art.select_one("p.instock.availability").get_text(strip=True) if art.select_one("p.instock.availability") else ""
        rating_p = art.select_one("p.star-rating")
        rating = ""
        if rating_p:
            for c in rating_p.get("class", []):
                if c != "star-rating":
                    rating = c
        book_links.append({
            "title": title,
            "product_page": product_link,
            "price": price,
            "availability": availability,
            "rating": rating
        })
    next_btn = soup.select_one("li.next a")
    if next_btn:
        page_url = urljoin(page_url, next_btn["href"])
    else:
        page_url = None

print("Found book entries:", len(book_links))

Found book entries: 1000


In [ ]:
sem = asyncio.Semaphore(50)

async def fetch_description(session: ClientSession, entry):
    url = entry["product_page"]
    try:
        async with sem:
            async with session.get(url, timeout=20) as resp:
                text = await resp.text()
    except Exception as e:
        entry["description"] = ""
        return
    sp = BeautifulSoup(text, "html.parser")
    desc = ""
    desc_tag = sp.select_one("#product_description")
    if desc_tag:
        next_p = desc_tag.find_next_sibling("p")
        if next_p:
            desc = next_p.get_text(strip=True)
    entry["description"] = desc

async def main_fetch(all_entries):
    conn = aiohttp.TCPConnector(limit=100)
    timeout = aiohttp.ClientTimeout(total=60)
    async with aiohttp.ClientSession(connector=conn, timeout=timeout) as session:
        tasks = [fetch_description(session, e) for e in all_entries]
        # Gather in chunks to reduce memory pressure if many tasks
        chunk_size = 200
        for i in range(0, len(tasks), chunk_size):
            await asyncio.gather(*tasks[i:i+chunk_size])

In [ ]:
await main_fetch(book_links)

In [ ]:
df = pd.DataFrame(book_links)
print("Final rows:", df.shape[0])
display(df.head())
csv_path = "/content/books_concurrent.csv"
df.to_csv(csv_path, index=False, encoding="utf-8-sig")
print("Saved to", csv_path)

Final rows: 1000


,title,product_page,price,availability,rating,description
0,A Light in the Attic,https://books.toscrape.com/catalogue/a-light-i...,Â£51.77,In stock,Three,It's hard to imagine a world without A Light i...
1,Tipping the Velvet,https://books.toscrape.com/catalogue/tipping-t...,Â£53.74,In stock,One,"""Erotic and absorbing...Written with starling ..."
2,Soumission,https://books.toscrape.com/catalogue/soumissio...,Â£50.10,In stock,One,"Dans une France assez proche de la nôtre, un h..."
3,Sharp Objects,https://books.toscrape.com/catalogue/sharp-obj...,Â£47.82,In stock,Four,"WICKED above her hipbone, GIRL across her hear..."
4,Sapiens: A Brief History of Humankind,https://books.toscrape.com/catalogue/sapiens-a...,Â£54.23,In stock,Five,From a renowned historian comes a groundbreaki...


Saved to /content/books_concurrent.csv
